# 词袋模型

 将所有词语装进一个袋子里，不考虑其词法和语序的问题，即每个词语都是独立的。
 
 例句:

Jane wants to go to Shenzhen.

Bob  wants to go to Shanghai.

根据上面的两句制作袋子，里包括Jane、wants、to、go、Shenzhen、Bob、Shanghai。假设建立一个数组（或词典）用于映射匹配。  
词典：[Jane, wants, to, go, Shenzhen, Bob, Shanghai]

根据上面的词典，对例句建立词袋模型(统计个数)如下：  
例句1 ： [1,1,2,1,1,0,0]  
例句2 ： [0,1,2,1,0,1,1]  

根据上面的词典，对例句建立词袋模型(不统计个数)如下：  
例句1 ： [1,1,1,1,1,0,0]  
例句2 ： [0,1,1,1,0,1,1]  

### 使用keras建立词袋模型

In [1]:
from keras.preprocessing.text import Tokenizer

samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# i创建一个分词器（tokenizer），设置为只考虑前1000个最常见的单词
tokenizer = Tokenizer(num_words=1000)

# 构建索引单词
tokenizer.fit_on_texts(samples)

# 将字符串转换为整数索引组成的列表
sequences = tokenizer.texts_to_sequences(samples)

one_hot_results = tokenizer.texts_to_matrix(samples, mode="binary")
print(one_hot_results)

# 得到单词索引
word_index = tokenizer.word_index  
print(word_index)

Using TensorFlow backend.


[[0. 1. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
{'the': 1, 'cat': 2, 'sat': 3, 'on': 4, 'mat': 5, 'dog': 6, 'ate': 7, 'my': 8, 'homework': 9}


### 使用nltk建立词袋模型

In [ ]:
import nltk
from nltk import FreqDist

corpus = 'this is my sentence ' \
 'this is my life ' \
 'this is the day'

tokens = nltk.word_tokenize(corpus)

# 统计词频
fdist = FreqDist(tokens)
# 打印'is'出现的次数
print(fdist['is'])

# 列举出现最多的50个词,虽然句子没有50个词
standard_freq_vector = fdist.most_common(50)
# 统计实际多少个词
size = len(standard_freq_vector)
print(standard_freq_vector)

# 按照出来的频率记录每个单词的位置

def position_lookup(v):
    res = {}
    counter = 0
    for word in v:
        res[word[0]] = counter
        counter += 1
    return res

# 生成standard_freq_vector的位置字典
standard_position_dict = position_lookup(standard_freq_vector)
print(standard_position_dict)

# 新句子
sentence = 'this is cool'
# 先新建⼀个跟我们的词典同样⼤⼩的向量
freq_vector = [0] * size
tokens = nltk.word_tokenize(sentence)

for word in tokens:
    try:
        freq_vector[standard_position_dict[word]] += 1
    except KeyError: #如果遇到新词则进入,如cool
        continue

print(freq_vector)

### 使用普通python语法对诗句进行词袋模型的建立

In [56]:
import collections

# 设置词向量的空间，即每首诗的维度
max_words = 1000


# 读取txt中的诗句，将诗的题目去除
# poetrys为列表类型，每个成员为每首诗
poetrys = [line.strip().replace(' ', '').split(':')[1] for line in open('../corpus/poetry.txt', encoding='utf-8')]

# 将所有诗句分割成单个词放入列表words中
words = []
for poetry in poetrys:
    words += [word for word in poetry]

# 统计所有词频, counter为字典类型，key=单个词， value为单个词出现的次数
counter = collections.Counter(words)

# 按照从大到小的顺序排名
count_pairs = sorted(counter.items(), key=lambda x: -x[1])

# words为元组类型
words, _ = zip(*count_pairs)


# 取出最多1000个词组成字典, 并将最后一个词设置为*, 即不在字典中的字用'*'代替
words_size = min(max_words, len(words))
words = words[:words_size] + ('*',)
words_size = len(words)

# 将字符映射成字典， key为词， value为词的编号0-1000
char2id_dict = {w: i for i, w in enumerate(words)}

# 打印未知字符'*'的编号 为 1000
unknow_char = char2id_dict.get('*')

# 定义char2id函数，用于查找字符的索引，找不到则返回1000
char2id = lambda char: char2id_dict.get(char, unknow_char)

# 根据诗句的长度进行排序
poetrys = sorted(poetrys, key=lambda line: len(line))

# 将诗句使用字典的索引将汉字替换成数字
poetrys_index_vector = [list(map(char2id, poetry)) for poetry in poetrys]

# 每首诗使用维度为1000的向量表示，在字典对应位置使用词频表示
# 并append到poetrys_vector列表中
poetrys_vector = []
for poetry in poetrys:
    poetry_vector = [0]*words_size
    for word in poetry:
        try:
            poetry_vector[char2id_dict[word]] += 1
        except KeyError: #如果遇到新词则进入
            poetry_vector[unknow_char] += 1
    poetrys_vector.append(poetry_vector)

# gensim

In [ ]:
from gensim import corpora, models, similarities
import jieba


"""
载入中文数据以及对应的包，corpora是构造词典的, similarities求相似性可以用得到
"""

raw_documents = [  
    '0无偿居间介绍买卖毒品的行为应如何定性',  
    '1吸毒男动态持有大量毒品的行为该如何认定',  
    '2如何区分是非法种植毒品原植物罪还是非法制造毒品罪',  
    '3为毒贩贩卖毒品提供帮助构成贩卖毒品罪',  
    '4将自己吸食的毒品原价转让给朋友吸食的行为该如何认定',  
    '5为获报酬帮人购买毒品的行为该如何认定',  
    '6毒贩出狱后再次够买毒品途中被抓的行为认定',  
    '7虚夸毒品功效劝人吸食毒品的行为该如何认定',  
    '8妻子下落不明丈夫又与他人登记结婚是否为无效婚姻',  
    '9一方未签字办理的结婚登记是否有效',  
    '10夫妻双方1990年按农村习俗举办婚礼没有结婚证 一方可否起诉离婚',  
    '11结婚前对方父母出资购买的住房写我们二人的名字有效吗',  
    '12身份证被别人冒用无法登记结婚怎么办？',  
    '13同居后又与他人登记结婚是否构成重婚罪',  
    '14未办登记只举办结婚仪式可起诉离婚吗',  
    '15同居多年未办理结婚登记，是否可以向法院起诉要求离婚'  
]
# 分词
texts = [[word for word in jieba.cut(document, cut_all=True)] for document in raw_documents]

# 建立词典
dictionary = corpora.Dictionary(texts)
print("The length of dictionary was ", len(dictionary))

# 根据词典，对当前语句中的词进行词频统计 即bag-of-words
corpus = [dictionary.doc2bow(text) for text in texts]

# 建立TFIDF
tfidf = models.TfidfModel(corpus)

# 根据模型计算语料库的TFIDF
corpus_tfidf = tfidf[corpus]

for i, corpus in enumerate(corpus_tfidf):
    print("The content of corpus_tifdf[{}] was {}".format(i, corpus))
    print("The length of corpus_tfidf[{}] was {}".format(i, len(corpus)))
    print("-----------------------------------")



# 创建相似度矩阵 将每个句子生成一个长度为词典大小空间的向量 index.index
index = similarities.MatrixSimilarity(corpus_tfidf)

# 显示第二个句子的词向量
print(index.index[1])